In [2]:
import pandas as pd
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer,BertModel
import logging
import difflib

`data_withGoldenLabel`为人工标注标签

`data_withSilverLabel`为算法标注标签

`data`去重:重复数据直接删除

将`author`中`nan`替换为`'Not Available'`

In [3]:
data = pd.read_table("./DataSet/book/book.txt" , sep='\t' , header=None , names=['source','isbn','name','author'])
data = data.drop_duplicates().reset_index(drop=True)
data.fillna('Not Available', inplace=True)
data['encode']=None
GoldenLabel = pd.read_table("./DataSet/book/book_golden.txt" , sep='\t' , header=None , names=['isbn','author'])
SilverLabel = pd.read_table("./DataSet/book/book_silver.txt" , sep='\t' , header=None , names=['isbn','author'])
SilverLabel = SilverLabel[SilverLabel['isbn']!='F']

加载`bert`模型和分词器

In [3]:
logging.basicConfig(level=logging.INFO)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\zxr\.pytorch_pretrained_bert\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\zxr\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:\Users\zxr\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\zxr\AppData\Local\Temp\tmp3mm1dwlh
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_pr

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [ ]:
'''
放弃这种处理方式

此处逐条处理数据，后期考虑封装成`Processor`

将每条数据的三部分拼接向量化还是分别向量化？

选用`[CLS]`的最后一个隐层编码

GPU内存不足，暂定在CPU上运行

for i in range(0,len(data)):
    item = data.loc[i]
    text = '[CLS] ' + (item['source']) + ' [SEP] ' + (item['name']) + ' [SEP] ' + (item['author']) + ' [SEP]'
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    #torch.cuda.empty_cache()
    #model.to('cuda')
    #tokens_tensor = tokens_tensor.to('cuda')
    with torch.no_grad():
        encoded_layers,_ = model(tokens_tensor)
    data.loc[i]['encode'] = encoded_layers[11][0][0]
'''

In [51]:
#尝试使用bert分别提取各列语义特征作为GCN的输入feature
data_golden = pd.read_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' )
def feedColumnIntoBert(feature):
    for i in range(0,len(feature)):
        feature.loc[i] = '[CLS] '+feature.loc[i]+' [SEP]'
        feature.loc[i] = tokenizer.tokenize(feature.loc[i])
        feature.loc[i] = tokenizer.convert_tokens_to_ids(feature.loc[i])
        feature.loc[i] = torch.tensor([feature.loc[i]])
        with torch.no_grad():
            encoded_layers,_ = model(feature.loc[i])
        feature.loc[i] = encoded_layers[-1][0][0]+encoded_layers[-2][0][0]+encoded_layers[-3][0][0]+encoded_layers[-4][0][0]
    return feature
feature1 = feedColumnIntoBert(pd.Series(data_golden['source']).copy())
feature2 = feedColumnIntoBert(pd.Series(data_golden['name']).copy())
feature3 = feedColumnIntoBert(pd.Series(data_golden['author']).copy())
for i in range(0,len(feature1)):
    feature1.loc[i] = torch.cat((feature1.loc[i],feature2.loc[i],feature3.loc[i]))
torch.save( feature1 , './DataSet/book/golden/claims_golden_encode_v2.pt' )

In [ ]:
#存储中间结果
torch.save( data['encode'] , './DataSet/book/claims_encode.pt' )
data['encode'] = None
data.to_csv( './DataSet/book/claims.txt' , sep='\t' , index=False )

#读取中间结果
data = None
data = pd.read_csv( "./DataSet/book/claims.txt" , sep='\t' )
#n_data['source'] = n_data['source'].astype(str)
#n_data['isbn'] = n_data['isbn'].astype(str)
#n_data['name'] = n_data['name'].astype(str)
#n_data['author'] = n_data['author'].astype(str)
data['encode'] = torch.load("./DataSet/book/claims_encode.pt")
GoldenLabel = pd.read_table("./DataSet/book/book_golden.txt" , sep='\t' , header=None , names=['isbn','author'])
SilverLabel = pd.read_table("./DataSet/book/book_silver.txt" , sep='\t' , header=None , names=['isbn','author'])

In [7]:
#1258 -> 1190
def sim_Jaccard (str1,str2) :
    set1 = set( str1.lower().replace(';',' ').replace(',',' ').replace('.',' ').replace(':',' ').replace('&',' ').
               replace('/',' ').replace('\'',' ').replace('(author)',' ').replace('(joint author)',' ').split() )
    set2 = set( str2.lower().replace(';',' ').replace(',',' ').replace('.',' ').replace(':',' ').replace('&',' ').
               replace('/',' ').replace('\'',' ').replace('(author)',' ').replace('(joint author)',' ').split() )
    return len(set1&set2)/len(set1|set2)

In [3]:
#观察数据信息
data_golden = pd.read_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' )
source_list = data_golden['source'].unique()
max_len = 0
min_len = 100
for i in range(len(source_list)):
    data_slice = data_golden[data_golden['source']==source_list[i]]
    temp_len = len(data_slice)
    max_len = max(max_len,temp_len)
    min_len = min(min_len,temp_len)

In [18]:
#使用Jaccard度量重新打标签
data_golden = pd.read_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' )
GoldenLabel = pd.read_table("./DataSet/book/book_golden.txt" , sep='\t' , header=None , names=['isbn','author'])

data_golden2 = pd.DataFrame()
for index1,row1 in GoldenLabel.iterrows():
    str1 = row1['author']
    data_slice = pd.DataFrame(data_golden[data_golden['isbn']==row1['isbn']])
    data_slice['label']=None
    
    for index2,row2 in data_slice.iterrows():
        str2 = row2['author']
        data_slice.loc[index2,'label'] = ( sim_Jaccard(str1,str2)>=0.8 )
            
    data_golden2 = data_golden2.append(data_slice)
data_golden2.reset_index(drop=True,inplace=True)

data_golden2.to_csv( './DataSet/book/golden/claims_golden2.txt' , sep='\t' , index=False )

In [100]:
#使用sklearn的TfidfVectorizer生成特征向量来比较bert的效果，发现比bert好很多？
#重新实验度量之后发现没有好很多
from sklearn.feature_extraction.text import TfidfVectorizer
data_golden.reset_index(drop=True,inplace=True)
vectorizer = TfidfVectorizer(min_df=0.01)

#2181*83
vectorizer.fit(data_golden['source'])
V1 = vectorizer.transform(data_golden['source']).toarray()
print('shape of V1:',V1.shape)

#2181*288
vectorizer.fit(data_golden['name'])
V2 = vectorizer.transform(data_golden['name']).toarray()
print('shape of V2:',V2.shape)

#2181*234
vectorizer.fit(data_golden['author'])
V3 = vectorizer.transform(data_golden['author']).toarray()
print('shape of V3:',V3.shape)

feature = np.concatenate((V1,V2,V3),axis=1)
feature = torch.Tensor(feature)
data_golden['encode']=None
for i in range(0,len(data_golden)):
    data_golden.loc[i,'encode'] = feature[i]
#torch.save( data_golden['encode'] , './DataSet/book/golden/claims_golden_encode_tfidf.pt' )

shape of V1: (2181, 49)
shape of V2: (2181, 181)
shape of V3: (2181, 107)


In [ ]:
#存储中间结果
data_golden.reset_index(drop=True,inplace=True)
torch.save( data_golden['encode'] , './DataSet/book/golden/claims_golden_encode.pt' )
data_golden['encode'] = None
data_golden.to_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' , index=False )

In [36]:
#读取中间结果
data_golden = pd.read_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' )
data_golden['encode'] = torch.load('./DataSet/book/golden/claims_golden_encode_v2.pt')

In [102]:
#查看bert分别提取的词向量是否存在问题，比全句直接提取好一点
data_slice = data_golden[ data_golden['isbn'] == data_golden.loc[0]['isbn'] ]
t1 = data_slice.iloc[0]['encode']
print(data_slice.iloc[0]['author'])
for index,row in data_slice.iterrows():
    t2 = row['encode']
    print(row['author'],(t1-t2).norm(2))

O'Leary, Timothy J.; O'Leary, Linda I.
O'Leary, Timothy J.; O'Leary, Linda I. tensor(0.)
O'Leary, Timothy J.; O'Leary, Linda I. tensor(1.)
Oleary tensor(0.6273)
OLEARY tensor(1.)
O'LEARY tensor(1.2333)
Timothy J O'Leary; Linda I O'Leary tensor(0.7890)
O'LEARY tensor(1.2333)
O'LEARY tensor(0.7218)
O'Leary, Timothy J.; O'Leary, Linda I. tensor(0.)
Timothy J Oandapos;Leary, Linda I Oandapos;Leary tensor(1.2738)
O'Leary, Timothy J.; O'Leary, Linda I. tensor(1.)
O'Leary, Timothy J./ O'Leary, Linda I. tensor(1.)
O'Leary, Timothy J, and O'Leary, Linda I tensor(0.7218)
Timothy J O'Leary; Linda I O'Leary tensor(0.7890)
O'Leary, Timothy J.; O'Leary, Linda I. tensor(1.)
O'Leary, Timothy J./ O'Leary, Linda I. tensor(1.2333)
O'Leary, Linda I. J. tensor(1.)


In [18]:
#查看bert直接提取的全句词向量是否存在问题，结果可能是有
for i in range(0,100):
    a = data_golden.loc[i]['encode']
    b = data_golden.loc[i+1]['encode']
    print(data_golden.loc[i])
    print(data_golden.loc[i+1])
    print(sum(a),sum(b),(a-b).norm(2),'\n')

source                                           ProService
isbn                                          9780073516677
name                                   Computing Essentials
author               O'Leary, Timothy J.; O'Leary, Linda I.
encode    [tensor(-2.9803), tensor(-2.7744), tensor(-2.0...
label                                                  True
Name: 0, dtype: object
source                               Collegebooksdirect.com
isbn                                          9780073516677
name                                   Computing Essentials
author               O'Leary, Timothy J.; O'Leary, Linda I.
encode    [tensor(-0.2487), tensor(-2.1276), tensor(-1.0...
label                                                  True
Name: 1, dtype: object
tensor(-213.0678) tensor(-207.7377) tensor(42.4367) 

source                               Collegebooksdirect.com
isbn                                          9780073516677
name                                   Computing Essentials


tensor(-207.1902) tensor(-207.9890) tensor(62.9375) 

source                                  Movies With A Smile
isbn                                          9780073516677
name            Computing Essentials 2007, Complete Edition
author             O'Leary, Timothy J, and O'Leary, Linda I
encode    [tensor(0.5335), tensor(-1.3453), tensor(-1.12...
label                                                  True
Name: 12, dtype: object
source                                        Players Quest
isbn                                          9780073516677
name      Computing Essentials 2007, Complete Edition (O...
author                   Timothy J O'Leary; Linda I O'Leary
encode    [tensor(-4.1352), tensor(-1.1925), tensor(-1.0...
label                                                  True
Name: 13, dtype: object
tensor(-207.9890) tensor(-207.9521) tensor(59.1789) 

source                                        Players Quest
isbn                                          9780073516677
name

tensor(-201.3535) tensor(-205.8448) tensor(45.0942) 

source                                           Bobs Books
isbn                                          9780072999389
name        Computer Accounting Essentials Using ePeachtree
author                         Yacht, Carol; Crosson, Susan
encode    [tensor(-1.8695), tensor(-2.1089), tensor(-1.1...
label                                                  True
Name: 22, dtype: object
source                                     Books Down Under
isbn                                          9780072999389
name        Computer Accounting Essentials Using ePeachtree
author                         Yacht, Carol; Crosson, Susan
encode    [tensor(-0.6274), tensor(-0.7345), tensor(-0.8...
label                                                  True
Name: 23, dtype: object
tensor(-205.8448) tensor(-201.7446) tensor(39.1784) 

source                                     Books Down Under
isbn                                          9780072999389
name

tensor(-224.1638) tensor(-216.7955) tensor(78.1187) 

source                                           Alinonline
isbn                                          9780072922363
name                Fundamentals of Finite Element Analysis
author                                     Hutton, David W.
encode    [tensor(-2.4856), tensor(-2.0434), tensor(-2.0...
label                                                  True
Name: 32, dtype: object
source                                      LGTextbooks.com
isbn                                          9780072922363
name                Fundamentals of Finite Element Analysis
author                                               HUTTON
encode    [tensor(0.8231), tensor(-2.6314), tensor(-0.86...
label                                                 False
Name: 33, dtype: object
tensor(-216.7955) tensor(-213.9319) tensor(73.5622) 

source                                      LGTextbooks.com
isbn                                          9780072922363
name

tensor(-213.8579) tensor(-212.4154) tensor(53.3630) 

source                                   Books2Anywhere.com
isbn                                          9780072922363
name                Fundamentals of Finite Element Analysis
author                                     Hutton, David V.
encode    [tensor(0.9839), tensor(-2.3735), tensor(-0.60...
label                                                  True
Name: 42, dtype: object
source                                        Papamedia.com
isbn                                          9780072922363
name                Fundamentals of Finite Element Analysis
author                        David V. Hutton, David Hutton
encode    [tensor(0.8313), tensor(-2.3892), tensor(-1.42...
label                                                 False
Name: 43, dtype: object
tensor(-212.4154) tensor(-202.7163) tensor(55.7175) 

source                                        Papamedia.com
isbn                                          9780072922363
name

tensor(-210.0692) tensor(-207.6974) tensor(47.2586) 

source                                               Caiman
isbn                                          9780072843996
name                          MICROSOFT WINDOWS XP COMPLETE
author                                       ESTEVEZ, EFREN
encode    [tensor(-4.3770), tensor(-1.6198), tensor(-1.9...
label                                                 False
Name: 52, dtype: object
source                                       Mellon's Books
isbn                                          9780072843996
name                          MICROSOFT WINDOWS XP COMPLETE
author                                        HAAG, STEPHEN
encode    [tensor(-2.1292), tensor(-1.2109), tensor(-2.0...
label                                                 False
Name: 53, dtype: object
tensor(-207.6974) tensor(-208.2973) tensor(49.3601) 

source                                       Mellon's Books
isbn                                          9780072843996
name

tensor(-206.6174) tensor(-203.0566) tensor(38.0800) 

source                                    Revaluation Books
isbn                                          9780072843996
name                          Microsoft Windows Xp Complete
author    Haag, Stephen/ Perry, James T./ Sosinsky, Barr...
encode    [tensor(-2.6642), tensor(-2.2452), tensor(-0.8...
label                                                  True
Name: 62, dtype: object
source                                           Cobain LLC
isbn                                          9780072466850
name              Introduction to the Finite Element Method
author                                          J. N. Reddy
encode    [tensor(-2.3870), tensor(-0.3540), tensor(-2.2...
label                                                  True
Name: 63, dtype: object
tensor(-203.0566) tensor(-218.2256) tensor(71.7002) 

source                                           Cobain LLC
isbn                                          9780072466850
name

tensor(-224.5855) tensor(-201.4934) tensor(80.9295) 

source                                 www.textbooksrus.com
isbn                                          9780072466850
name      An Introduction to the Finite Element Method (...
author                                          J. N. Reddy
encode    [tensor(0.0781), tensor(-1.8425), tensor(-1.39...
label                                                  True
Name: 72, dtype: object
source                     D & A Worldwide Textbook Service
isbn                                          9780072466850
name              Introduction To The Finite Element Method
author                                          Reddy, J.N.
encode    [tensor(-2.0981), tensor(-1.0404), tensor(-1.4...
label                                                  True
Name: 73, dtype: object
tensor(-201.4934) tensor(-214.6785) tensor(63.2741) 

source                     D & A Worldwide Textbook Service
isbn                                          9780072466850
name

tensor(-213.9699) tensor(-216.2072) tensor(56.4343) 

source                                            Indoo.com
isbn                                          9780072466850
name           An Introduction to the Finite Element Method
author                                                REDDY
encode    [tensor(0.8534), tensor(-1.9232), tensor(-1.87...
label                                                 False
Name: 82, dtype: object
source                                         Missionbooks
isbn                                          9780072466850
name              Introduction To The Finite Element Method
author                                          Reddy, J.N.
encode    [tensor(-3.1257), tensor(-1.2700), tensor(-1.2...
label                                                  True
Name: 83, dtype: object
tensor(-216.2072) tensor(-222.6943) tensor(64.0914) 

source                                         Missionbooks
isbn                                          9780072466850
name

tensor(-207.0382) tensor(-213.6114) tensor(61.7605) 

source                                         textbooksNow
isbn                                          9780072255096
name                       Prin of Computer Security (w/CD)
author                                              Conklin
encode    [tensor(0.2385), tensor(0.4502), tensor(-2.184...
label                                                 False
Name: 93, dtype: object
source                                       Mellon's Books
isbn                                          9780072255096
name                        PRINCIPLES OF COMPUTER SECURITY
author                                  CONKLIN, WM. ARTHUR
encode    [tensor(-2.1292), tensor(-1.2109), tensor(-2.0...
label                                                 False
Name: 94, dtype: object
tensor(-213.6114) tensor(-214.2999) tensor(69.0369) 

source                                       Mellon's Books
isbn                                          9780072255096
name

In [4]:
#这里讨巧使用`difflib`来度量字符串相似度，相似度的阈值后期可调，发现字符串肉眼相似但相似度量极低的情况！考虑第三方库的字符串相似度量？
#后续可能需要改进，考虑再次引入`bert`
data['label'] = None
data['label'] = data['label'].astype(bool)
data_golden = pd.DataFrame(data.drop(data.index,inplace=False))

for index1,row1 in GoldenLabel.iterrows():
    str1 = row1['author'].lower()
    data_goldenSlice = pd.DataFrame(data[data['isbn']==row1['isbn']])
    for index2,row2 in data_goldenSlice.iterrows():
        str2 = row2['author'].lower()
        data_goldenSlice.loc[index2,'label'] = (difflib.SequenceMatcher(None,str1,str2).quick_ratio()>0.8)
    data_golden = data_golden.append(data_goldenSlice)

In [47]:
#尝试不同的字符串相似度比较方法，没有进展 -> Jaccard度量或许强于编辑距离
data_golden = pd.read_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' )
GoldenLabel = pd.read_table("./DataSet/book/book_golden.txt" , sep='\t' , header=None , names=['isbn','author'])
test_slice = pd.DataFrame(data_golden[data_golden.isbn==GoldenLabel.loc[99]['isbn']])
test_answer = GoldenLabel.loc[99]
import difflib
import Levenshtein
str2 = test_answer['author'].lower()
print(str2)
for index,row in test_slice.iterrows():
    str1 = row['author'].lower()
    print(str1)
    #seq = difflib.SequenceMatcher(lambda x:x in {' ',',',';',':','(',')','author'}, str1,str2)
    #seq = difflib.SequenceMatcher(None, str1,str2)
    #sim1 = seq.ratio()
    #sim2 = Levenshtein.distance(str1, str2)
    Edit_sim = Levenshtein.ratio(str1, str2)
    #sim4 = Levenshtein.jaro(str1, str2 )
    #sim5 = Levenshtein.jaro_winkler(str2 , str1 )
    Jacc_sim = sim_Jaccard(str1,str2)
    print('Edit_sim:',Edit_sim,' Jacc_sim:',Jacc_sim)

moore, martin ;  hancock, steven ;
moore, martin l
Edit_sim: 0.5714285714285714  Jacc_sim: 0.4
martin moore, steven hancock
Edit_sim: 0.5483870967741935  Jacc_sim: 1.0
moore
Edit_sim: 0.2564102564102564  Jacc_sim: 0.25
martin moore
Edit_sim: 0.391304347826087  Jacc_sim: 0.5
moore, martin
Edit_sim: 0.5531914893617021  Jacc_sim: 0.5
moore
Edit_sim: 0.2564102564102564  Jacc_sim: 0.25
moore, martin (author) hancock, steven (author) hancock, steve (joint author)
Edit_sim: 0.5765765765765766  Jacc_sim: 0.8
moore, martin l./ hancock, steven
Edit_sim: 0.8955223880597015  Jacc_sim: 0.8
moore
Edit_sim: 0.2564102564102564  Jacc_sim: 0.25
moore
Edit_sim: 0.2564102564102564  Jacc_sim: 0.25
moore, martin (author) hancock, steven (author) hancock, steve (joint author)
Edit_sim: 0.5765765765765766  Jacc_sim: 0.8
moore
Edit_sim: 0.2564102564102564  Jacc_sim: 0.25
moore, martin alan
Edit_sim: 0.6153846153846154  Jacc_sim: 0.4


In [21]:
#双重验证考虑一下？作为后期工作，暂且告一段落
GoldenLabel = pd.read_table("./DataSet/book/book_golden.txt" , sep='\t' , header=None , names=['isbn','author'])

data_golden1 = pd.read_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' )
data_golden2 = pd.read_csv( './DataSet/book/golden/claims_golden2.txt' , sep='\t' )
print('sum of EditLabel : ',sum(data_golden1['label']))
print('sum of JaccLabel : ',sum(data_golden2['label']))
difflabel = data_golden1['label']^data_golden2['label']
print('sum of DiffLabel : ',sum(difflabel))
print('sum of OpenLabel : ',sum(data_golden1['label']|data_golden2['label']))
print('sum of ClosLabel : ',sum(data_golden1['label']&data_golden2['label']))

for index,row in data_golden2[difflabel].iterrows():
    print(row)
    print(GoldenLabel[GoldenLabel['isbn']==row['isbn']])

sum of EditLabel :  1258
sum of JaccLabel :  1190
sum of DiffLabel :  264
sum of OpenLabel :  1356
sum of ClosLabel :  1092
source           Limelight Bookshop
isbn                  9780073516677
name      Computing Essentials 2007
author         O'Leary, Linda I. J.
encode                          NaN
label                          True
Name: 16, dtype: object
            isbn                                      author
0  9780073516677  o'leary, timothy j.;  o'leary, linda i.;  
source                      Reliable Book Service
isbn                                9780072922363
name      Fundamentals of Finite Element Analysis
author                           Hutton, David W.
encode                                        NaN
label                                       False
Name: 27, dtype: object
            isbn               author
2  9780072922363  hutton, david v.;  
source                                 Alinonline
isbn                                9780072922363
name      Fund

12  1558609342  kumar, c bala;  kline, paul j.;  thompson, tim...
source                                  Movies With A Smile
isbn                                             1558609342
name      Bluetooth Application Programming with the Jav...
author    Kumar, C Bala, and Kline, Paula J, and Thompso...
encode                                                  NaN
label                                                 False
Name: 241, dtype: object
          isbn                                             author
12  1558609342  kumar, c bala;  kline, paul j.;  thompson, tim...
source                                         Quartermelon
isbn                                             1558609342
name      Bluetooth Application Programming with the Jav...
author    Kumar, C. Bala (Author) Kline, Paul (Author) T...
encode                                                  NaN
label                                                  True
Name: 255, dtype: object
          isbn                  

          isbn                                            author
24  1558606483  puder, arno ;  romer, kay ;  pilhofer, frank ;  
source                                      AHA-BUCH
isbn                                      1558606483
name                Distributed Systems Architecture
author    Arno Puder, Kay R&ouml;mer, Frank Pilhofer
encode                                           NaN
label                                          False
Name: 506, dtype: object
          isbn                                            author
24  1558606483  puder, arno ;  romer, kay ;  pilhofer, frank ;  
source                                   Revaluation Books
isbn                                            1558605959
name                                    Swarm Intelligence
author    Kennedy, James F./ Eberhart, Russell/ Shi, Yuhui
encode                                                 NaN
label                                                False
Name: 546, dtype: object
          isbn    

          isbn                                       author
45  0672323435  mcmanus, jeffrey p.;  goldstein, jackie ;  
source                                          Sandy Chong
isbn                                             0672323435
name                  Database Access With Visual Basic.Net
author    Kevin T. Price, Jeffrey P. McManus, Jackie Gol...
encode                                                  NaN
label                                                 False
Name: 811, dtype: object
          isbn                                       author
45  0672323435  mcmanus, jeffrey p.;  goldstein, jackie ;  
source                                    Revaluation Books
isbn                                             0672323435
name              Database Access with Visual Basic&reg;NET
author    McManus, Jeffrey P./ Goldstein, Jackie/ Price,...
encode                                                  NaN
label                                                 False
Name: 815, dtyp

53  032131283X  ben-ari, m. ;  
source                                        Players Quest
isbn                                             032131283X
name      Principles of Concurrent and Distributed Progr...
author                                           M. Ben-Ari
encode                                                  NaN
label                                                  True
Name: 1051, dtype: object
          isbn           author
53  032131283X  ben-ari, m. ;  
source                            Morgenstundt Buch & Kunst
isbn                                             0321304292
name      Internet Effectively: A Beginner's Guide to th...
author    Scollard, Lafayette; Scollard, Sharon; Adams, ...
encode                                                  NaN
label                                                  True
Name: 1062, dtype: object
          isbn                                author
54  0321304292  adams, tyrone ;  scollard, sharon ; 
source                    

68  0201823764  abrahams, paul w.;  larson, bruce r.;  
source                                   Indoo.com
isbn                                    0201776316
name      TCP/IP Illustrated Volumes 1-3 Boxed Set
author          Stevens, W. RichardWright, Gary R.
encode                                         NaN
label                                        False
Name: 1529, dtype: object
          isbn                                     author
69  0201776316  stevens, w. richard;  wright, gary r. ;  
source                         AHA-BUCH
isbn                         0201776316
name        TCP/IP Illustrated, 3 vols.
author    W. R. Stevens, Gary R. Wright
encode                              NaN
label                              True
Name: 1534, dtype: object
          isbn                                     author
69  0201776316  stevens, w. richard;  wright, gary r. ;  
source    Marando.de Versandbuchhandlung
isbn                          0201776316
name         TCP/IP Illustrated,

84  0201379236  musser, david r.;  derge, gillmer j.;  saini, ...
source                                     Indoo.com
isbn                                      0131872893
name                  Learning to Program with Alice
author    Dann, Wanda P.Cooper, StephenPausch, Randy
encode                                           NaN
label                                          False
Name: 1910, dtype: object
          isbn                                             author
87  0131872893  dann, wanda p.;  cooper, stephen ;  pausch, ra...
source                                   textbookxdotcom
isbn                                          0131872893
name       Learning to Program with Alice, Final Version
author    Dann, Wanda P.BRCooper, StephenBRPausch, Randy
encode                                               NaN
label                                              False
Name: 1913, dtype: object
          isbn                                             author
87  0131872893  dann, wa

Name: 2077, dtype: object
          isbn                                             author
94  0123704529  hornick, mark f.;  marcade, erik ;  venkayala,...
source                                      Annex Books Inc
isbn                                             0123704529
name      Java Data Mining: Strategy, Standard, and Prac...
author    Hornick, Mark F.; Marcad&eacute;, Erik; Venkay...
encode                                                  NaN
label                                                 False
Name: 2078, dtype: object
          isbn                                             author
94  0123704529  hornick, mark f.;  marcade, erik ;  venkayala,...
source                                       LAKESIDE BOOKS
isbn                                             0123704529
name      JAVA DATA MINING: STRATEGY, STANDARD, AND PRAC...
author        HORNICK, MARK F.MARCADE, ERIKVENKAYALA, SUNIL
encode                                                  NaN
label                   

In [ ]:
'''
book_silver.txt数据集有点问题，下一块代码暂且搁置

这个`book`数据集问题很大，值得吐槽:
`dataset`中存在大量冗余数据；
golden中存在两条`isbn`为`F`的奇怪数据；
`silver`中许多数据`isbn`和`dataset`对应不上

data['label'] = None
data['label'] = data['label'].astype(bool)
data_silver = pd.DataFrame(data.drop(data.index,inplace=False))

for index1,row1 in SilverLabel.iterrows():
    str1 = row1['author'].lower()
    data_silverSlice = pd.DataFrame(data[data['isbn']==row1['isbn']])
    for index2,row2 in data_silverSlice.iterrows():
        str2 = row2['author'].lower()
        data_silverSlice.loc[index2,'label'] = (difflib.SequenceMatcher(None,str1,str2).quick_ratio()>0.8)
    data_silver = data_silver.append(data_silverSlice)
'''